In [3]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shapley import ShapleyIndices, ShapleyKrigingIndices
from shapley.tests import Ishigami, AdditiveGaussian
from shapley.plots import set_style_paper, plot_sensitivity_results
from shapley.base import ProbabilisticModel 
from shapley.kriging import KrigingModel

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

savefigs = True
ylim = [0., 1.]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
correlation = ot.CorrelationMatrix(pd.read_csv('./data/correlation.csv', sep=',', header=None).values)
copula = ot.NormalCopula(correlation)

In [41]:
nu_betas = [247, 218, 148, 105]
sigma_betas = [6, 5, 4, 3]
nu_normal = [35, 18, 14, 11, 8, 1, -31]
sigma_normal = [5, 5, 5, 5, 5, 5, 5]
bounds_low = [235, 207, 141, 100]
bounds_up = [259, 229, 155, 110]

margins = []
for nu, sigma, a, b in zip(nu_betas, sigma_betas, bounds_low, bounds_up):
    dist = ot.Beta()
    dist.setParameter(ot.BetaMuSigma()([nu, sigma, a, b]))
    margins.append(dist)
    
for nu, sigma in zip(nu_normal, sigma_normal):
    margins.append(ot.Normal(nu, sigma))

In [43]:
input_distribution = ot.ComposedDistribution(margins, copula)

In [44]:
data = pd.read_csv('./data/results_unif.txt', sep=' ', header=None)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2.468000e+11,2.182000e+11,1.053000e+11,1.485000e+11,35.000000,18.000000,14.000000,11.000000,8.000000,1.000000,-31.000000,5.696650e-11,468.0,9.0
1,2.529700e+11,2.127450e+11,1.079325e+11,1.447875e+11,42.500000,10.500000,21.500000,3.500000,0.500000,8.500000,-38.500000,5.693590e-11,492.0,7.0
2,2.406300e+11,2.236550e+11,1.026675e+11,1.522125e+11,27.500000,25.500000,6.500000,18.500000,15.500000,-6.500000,-23.500000,7.235190e-11,481.0,9.0
3,2.437150e+11,2.154725e+11,1.066162e+11,1.429312e+11,46.250000,29.250000,2.750000,14.750000,-3.250000,12.250000,-34.750000,7.816420e-11,489.0,8.0
4,2.560550e+11,2.263825e+11,1.013512e+11,1.503562e+11,31.250000,14.250000,17.750000,-0.250000,11.750000,-2.750000,-19.750000,6.892650e-11,472.0,8.0
5,2.498850e+11,2.100175e+11,1.039838e+11,1.466438e+11,23.750000,21.750000,25.250000,22.250000,4.250000,-10.250000,-42.250000,8.455120e-11,539.0,0.0
6,2.375450e+11,2.209275e+11,1.092488e+11,1.540688e+11,38.750000,6.750000,10.250000,7.250000,19.250000,4.750000,-27.250000,7.051400e-11,486.0,8.0
7,2.390875e+11,2.141088e+11,1.033256e+11,1.512844e+11,36.875000,8.625000,0.875000,24.125000,-1.375000,-12.125000,-25.375000,7.587450e-11,506.0,5.0
8,2.514275e+11,2.250188e+11,1.085906e+11,1.438594e+11,21.875000,23.625000,15.875000,9.125000,13.625000,2.875000,-40.375000,8.303780e-11,482.0,9.0
9,2.575975e+11,2.086538e+11,1.059581e+11,1.549969e+11,29.375000,16.125000,23.375000,16.625000,6.125000,10.375000,-17.875000,6.665030e-11,474.0,8.0


In [ ]:
KrigingModel(model=lambda x: None, input_distribution=input_distribution)